<a href="https://colab.research.google.com/github/freitas-oa/Computer-Vision/blob/main/mixed_dataset_PPE_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ultralytics opencv-python roboflow numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [2]:
from roboflow import Roboflow
from ultralytics import YOLO
import os

# === 1. DOWNLOAD DATASET ===
rf = Roboflow(api_key="Y89fnR5Fz37PMRQ6ljyY")
project = rf.workspace("joseph-nelson").project("hard-hat-workers")
dataset = project.version(10).download("yolov8")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Hard-Hat-Workers-10 in yolov8:: 100%|██████████| 14082/14082 [00:01<00:00, 9268.08it/s] 


In [3]:
project = rf.workspace("object-detection-cwcjo").project("caps-gi4un")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Caps-2 in yolov8:: 100%|██████████| 5192/5192 [00:00<00:00, 8768.55it/s]


In [9]:

import os
import glob
import random

from ultralytics import YOLO

import cv2
from google.colab.patches import cv2_imshow


In [10]:
# Base paths for the two datasets
HARDHAT_ROOT = "/content/Hard-Hat-Workers-10"
CAPS_ROOT = "/content/Caps-2"

# Sanity checks (will raise an error if something is wrong)
assert os.path.isdir(HARDHAT_ROOT), f"Hard-Hat dataset not found at {HARDHAT_ROOT}"
assert os.path.isdir(CAPS_ROOT), f"Caps dataset not found at {CAPS_ROOT}"

print("Hard-Hat-Workers-10 structure:")
print(os.listdir(HARDHAT_ROOT))

print("\nCaps-2 structure:")
print(os.listdir(CAPS_ROOT))


Hard-Hat-Workers-10 structure:
['valid', 'data.yaml', 'test', 'train', 'README.dataset.txt', 'README.roboflow.txt']

Caps-2 structure:
['valid', 'data.yaml', 'test', 'train', 'README.dataset.txt', 'README.roboflow.txt']


In [11]:
def remap_caps_labels(root_dir: str, old_id: int = 0, new_id: int = 3):
    """
    Remap YOLO labels in Caps-2 dataset:
    - For every .txt in train/labels and valid/labels,
      change class id 'old_id' to 'new_id'.
    """
    label_dirs = [
        os.path.join(root_dir, "train", "labels"),
        os.path.join(root_dir, "valid", "labels"),
    ]

    for label_dir in label_dirs:
        assert os.path.isdir(label_dir), f"Label directory not found: {label_dir}"
        txt_files = glob.glob(os.path.join(label_dir, "*.txt"))

        print(f"Processing {len(txt_files)} label files in {label_dir} ...")

        for txt_path in txt_files:
            with open(txt_path, "r") as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                parts = line.split()
                # Replace class id if it matches old_id
                if parts[0] == str(old_id):
                    parts[0] = str(new_id)
                new_lines.append(" ".join(parts) + "\n")

            # Overwrite the file with remapped labels
            with open(txt_path, "w") as f:
                f.writelines(new_lines)

    print("Remapping complete.")


remap_caps_labels(CAPS_ROOT, old_id=0, new_id=3)


Processing 2041 label files in /content/Caps-2/train/labels ...
Processing 425 label files in /content/Caps-2/valid/labels ...
Remapping complete.


In [12]:
def list_images(folder: str):
    """
    Recursively list all image files in a folder (common extensions).
    Returns a sorted list of absolute paths.
    """
    exts = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".JPG", ".JPEG", ".PNG")
    paths = []
    for root, dirs, files in os.walk(folder):
        for fname in files:
            if fname.endswith(exts):
                paths.append(os.path.abspath(os.path.join(root, fname)))
    return sorted(paths)


# Hard-Hat images
hardhat_train_images = list_images(os.path.join(HARDHAT_ROOT, "train", "images"))
hardhat_val_images = list_images(os.path.join(HARDHAT_ROOT, "valid", "images"))

# Caps images
caps_train_images = list_images(os.path.join(CAPS_ROOT, "train", "images"))
caps_val_images = list_images(os.path.join(CAPS_ROOT, "valid", "images"))

print(f"Hard-Hat train images: {len(hardhat_train_images)}")
print(f"Hard-Hat valid images: {len(hardhat_val_images)}")
print(f"Caps train images:     {len(caps_train_images)}")
print(f"Caps valid images:     {len(caps_val_images)}")

# ---- Create train_combo.txt ----
train_combo = hardhat_train_images + hardhat_val_images + caps_train_images + caps_val_images
train_list_path = "/content/train_combo.txt"

with open(train_list_path, "w") as f:
    f.write("\n".join(train_combo))

print(f"\nCreated {train_list_path} with {len(train_combo)} image paths.")

# ---- Create val_combo.txt ----
# Option 1 (as requested): use ONLY Hard-Hat valid images
val_combo = hardhat_val_images

# If you want to ALSO include caps_val_images, uncomment this:
# val_combo = hardhat_val_images + caps_val_images

val_list_path = "/content/val_combo.txt"

with open(val_list_path, "w") as f:
    f.write("\n".join(val_combo))

print(f"Created {val_list_path} with {len(val_combo)} image paths.")


Hard-Hat train images: 4916
Hard-Hat valid images: 1413
Caps train images:     2041
Caps valid images:     425

Created /content/train_combo.txt with 8795 image paths.
Created /content/val_combo.txt with 1413 image paths.


In [13]:
names_dict = {
    0: "head",
    1: "helmet",
    2: "person",
    3: "cap",
}

data_yaml_path = "/content/data_multi.yaml"

with open(data_yaml_path, "w") as f:
    f.write('path: ""\n')
    f.write(f"train: {os.path.abspath(train_list_path)}\n")
    f.write(f"val: {os.path.abspath(val_list_path)}\n")
    f.write("nc: 4\n")
    f.write("names:\n")
    for k, v in names_dict.items():
        f.write(f"  {k}: {v}\n")

print("Created data YAML at:", data_yaml_path)

# Show the contents for confirmation
with open(data_yaml_path, "r") as f:
    print("\n---- data_multi.yaml ----")
    print(f.read())


Created data YAML at: /content/data_multi.yaml

---- data_multi.yaml ----
path: ""
train: /content/train_combo.txt
val: /content/val_combo.txt
nc: 4
names:
  0: head
  1: helmet
  2: person
  3: cap



In [14]:
# Load COCO-pretrained YOLOv8s model
model = YOLO("yolov8s.pt")

# Train with the combined dataset
train_results = model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,   # reduce if you hit CUDA OOM
    lr0=0.001,
    name="yolov8s_helmet_cap_multiclass",
)

print("Training finished.")


Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data_multi.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_helmet_cap_multiclass, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.